<h1>Correção de Dados Brutos</h1>

Esse notebook corrige os dados brutos de MPU-9250 e GPS. 

<h4>Seleção de Dataset</h4>

In [ ]:
# # Saveiro 1
# workFolder = '2019-12-24 17-21-12 - Saveiro Cenário 1'

# # Em segundos
# mpu_data_direita_shift_time = 0 
# mpu_data_esquerda_shift_time = 0 

In [ ]:
# # Saveiro 2
# workFolder = '2019-12-24 17-59-52 - Saveiro Cenário 2'

# # Em segundos
# mpu_data_direita_shift_time = 0 
# mpu_data_esquerda_shift_time = 0 

In [ ]:
# # Saveiro 3
# workFolder = '2019-12-24 18-37-13 - Saveiro Cenário 3'

# # Em segundos
# mpu_data_direita_shift_time = 0 
# mpu_data_esquerda_shift_time = 0 

In [ ]:
# # Bravo 1
# workFolder = '2019-12-25 17-44-48 - Bravo Cenário 1'

# # Em segundos
# mpu_data_direita_shift_time = 0 
# mpu_data_esquerda_shift_time = 0 

In [ ]:
# # Bravo 2
# workFolder = '2019-12-25 18-17-03 - Bravo Cenário 2'

# # Em segundos
# mpu_data_direita_shift_time = 0 
# mpu_data_esquerda_shift_time = 0 

In [ ]:
# # Bravo 3
# workFolder = '2019-12-25 18-44-19 - Bravo Cenário 3'

# # Em segundos
# mpu_data_direita_shift_time = 0 
# mpu_data_esquerda_shift_time = 0

In [ ]:
# # Palio 1
# workFolder = '2019-12-26 18-24-09 - Palio Cenário 1'

# # Em segundos
# mpu_data_direita_shift_time = 0 
# mpu_data_esquerda_shift_time = 0 

In [ ]:
# # Palio 2
# workFolder = '2019-12-26 18-51-03 - Palio Cenário 2'

# # Em segundos
# mpu_data_direita_shift_time = 0 
# mpu_data_esquerda_shift_time = 0 

In [ ]:
# # Palio 3
# workFolder = '2019-12-26 19-23-31 - Palio Cenário 3'

# # Em segundos
# mpu_data_direita_shift_time = 0 
# mpu_data_esquerda_shift_time = 0 

<h4>Importação de Pacotes</h4>

In [1]:
import os
import folium
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil import parser
from haversine import haversine, Unit
from geopy.distance import geodesic
from tqdm import tqdm, tqdm_notebook
pd.set_option("float_format", '{:0.10f}'.format)
pd.set_option('display.max_columns', 30)
%matplotlib inline

<h4>Definição de Parâmetros</h4>

In [2]:
# Barra de Progresso
load_bar = None

# Em segundos | 0.1s = 10Hz | 0.01s = 100Hz
sampling_rate_interval = 0.01 

# 1 = 0.1s = 10Hz | 2 = 0.01s = 100Hz
sampling_rate_interval_decimals = len(str(sampling_rate_interval).split(".")[1])

# Valor da gravidade em m/s²
gravity = 9.80665

<h4>Funções Utilitárias</h4>

In [3]:
# Transforma Date String em Timestamp Int
def parseDateToTimestamp(value):
    newDate = parser.parse(value)
    return datetime.timestamp(newDate)

# Transforma Timestamp Int em Date String 
def parseTimestampToDate(value):
    newDate = datetime.fromtimestamp(value)
    return newDate.strftime("%Y-%m-%dT%H:%M:%S")

# Formata o timestamp limitando casas decimais
def roundTimestamp(value):
    return round(value, sampling_rate_interval_decimals)

# Deslocamento para correção de delays/horários incorretos entre dispositivos
def shiftTime(data, time): # time in seconds
    data['timestamp'] = data['timestamp'] + time
    return data

# Cria a Pasta se Ainda Não Existir
def createFolder(path):
    
    if not os.path.exists(path):
        os.makedirs(path)
        
# Traduz o lado para inglês - nome de arquivo
def translateSide(side):
    return "right" if side == "direita" else "left"

<h4>Importação dos Dados Brutos</h4>

In [13]:
readFolder = 'E:\\Dados Brutos\\100 Hz\\' + workFolder + '\\'
saveFolder = 'E:\\Dados Processados\\100 Hz\\' + workFolder + '\\'
createFolder(saveFolder)

raw_gps = pd.read_csv(readFolder + 'gps.csv', float_precision="high")

raw_data_68_direita = pd.read_csv(readFolder + 'direita\\data-set-mpu-0x68.csv', delimiter=";", float_precision="high")
raw_data_69_direita = pd.read_csv(readFolder + 'direita\\data-set-mpu-0x69.csv', delimiter=";", float_precision="high")
settings_68_direita = pd.read_csv(readFolder + 'direita\\settings-mpu-0x68.csv', delimiter=";", float_precision="high")
settings_69_direita = pd.read_csv(readFolder + 'direita\\settings-mpu-0x69.csv', delimiter=";", float_precision="high")

raw_data_68_direita = shiftTime(raw_data_68_direita, mpu_data_direita_shift_time)
raw_data_69_direita = shiftTime(raw_data_69_direita, mpu_data_direita_shift_time)

raw_data_68_esquerda = pd.read_csv(readFolder + 'esquerda\\data-set-mpu-0x68.csv', delimiter=";", float_precision="high")
raw_data_69_esquerda = pd.read_csv(readFolder + 'esquerda\\data-set-mpu-0x69.csv', delimiter=";", float_precision="high")
settings_68_esquerda = pd.read_csv(readFolder + 'esquerda\\settings-mpu-0x68.csv', delimiter=";", float_precision="high")
settings_69_esquerda = pd.read_csv(readFolder + 'esquerda\\settings-mpu-0x69.csv', delimiter=";", float_precision="high")

raw_data_68_esquerda = shiftTime(raw_data_68_esquerda, mpu_data_esquerda_shift_time)
raw_data_69_esquerda = shiftTime(raw_data_69_esquerda, mpu_data_esquerda_shift_time)

data_gps = None
data_direita = None
data_esquerda = None

settings_direita = None
settings_esquerda = None

iniTime = roundTimestamp(max(
    raw_data_68_direita['timestamp'].min(), 
    raw_data_69_direita['timestamp'].min(), 
    raw_data_68_esquerda['timestamp'].min(),
    raw_data_69_esquerda['timestamp'].min(),
))

endTime = roundTimestamp(min(
    raw_data_68_direita['timestamp'].max(), 
    raw_data_69_direita['timestamp'].max(), 
    raw_data_68_esquerda['timestamp'].max(),
    raw_data_69_esquerda['timestamp'].max(),
))   

<h2>Correção de Dados de GPS</h2>

<h4>Visualização de Dados</h4>

In [14]:
raw_gps.describe()

,lat,lon,elevation,accuracy,bearing,speed,satellites,hdop,vdop,pdop,geoidheight,ageofdgpsdata,dgpsid,activity,battery,annotation
count,1034.0000000000,1034.0000000000,1018.0000000000,1034.0000000000,999.0000000000,1014.0000000000,1034.0000000000,1014.0000000000,1014.0000000000,1014.0000000000,1014.0000000000,0.0000000000,0.0000000000,0.0000000000,1034.0000000000,0.0000000000
mean,-27.6995172683,-51.1138419187,938.8248980923,4.7712108317,209.0205083810,10.2671825258,19.6557059961,0.8223865878,1.2480276134,1.5103550296,3.6000000000,nan,nan,nan,83.4487427466,nan
std,0.0119134474,0.0102876635,35.7602009235,3.0562150066,104.1766868883,8.0904605400,3.0372695192,0.0602874364,0.1094320581,0.1141571637,0.0000000000,nan,nan,nan,0.9568539961,nan
min,-27.7177921860,-51.1288512040,881.7107801621,4.0000000000,0.0417944800,0.0000000000,0.0000000000,0.7000000000,1.0000000000,1.3000000000,3.6000000000,nan,nan,nan,82.0000000000,nan
25%,-27.7128200832,-51.1236681052,904.0860102362,4.0000000000,127.0635300000,4.3545877500,20.0000000000,0.8000000000,1.2000000000,1.4000000000,3.6000000000,nan,nan,nan,83.0000000000,nan
50%,-27.6942399416,-51.1132811534,949.3335052092,4.0000000000,222.4177200000,8.0076310000,20.0000000000,0.8000000000,1.2000000000,1.5000000000,3.6000000000,nan,nan,nan,83.0000000000,nan
75%,-27.6889856886,-51.1043528981,976.1643486925,4.0000000000,307.2157350000,19.0852957500,21.0000000000,0.8000000000,1.3000000000,1.5000000000,3.6000000000,nan,nan,nan,84.0000000000,nan
max,-27.6857704000,-51.0988452291,1010.7376196344,36.9000000000,359.4611200000,25.5521180000,22.0000000000,1.3000000000,1.8000000000,2.2000000000,3.6000000000,nan,nan,nan,85.0000000000,nan


In [15]:
raw_gps.head()

,time,lat,lon,elevation,accuracy,bearing,speed,satellites,provider,hdop,vdop,pdop,geoidheight,ageofdgpsdata,dgpsid,activity,battery,annotation
0,2019-12-26T22:20:54.093Z,-27.7176688693,-51.0988452291,1010.7376196344,8.0000000000,196.8202800000,1.0591824000,0,gps,1.0000000000,1.7000000000,2.0000000000,3.6000000000,nan,nan,nan,85,nan
1,2019-12-26T22:20:55.092Z,-27.7176899184,-51.0989381528,1001.9054212869,6.0000000000,244.7717900000,1.1281757000,11,gps,0.9000000000,1.5000000000,1.8000000000,3.6000000000,nan,nan,nan,85,nan
2,2019-12-26T22:20:56.083Z,-27.7177004171,-51.0989531781,998.9896134512,6.0000000000,270.3939800000,0.2207288100,11,gps,0.9000000000,1.6000000000,1.8000000000,3.6000000000,nan,nan,nan,85,nan
3,2019-12-26T22:20:57.083Z,-27.7177001244,-51.0989532787,994.7213603738,8.0000000000,nan,0.0000000000,14,gps,0.8000000000,1.3000000000,1.6000000000,3.6000000000,nan,nan,nan,85,nan
4,2019-12-26T22:20:58.083Z,-27.7177001273,-51.0989534652,993.1997883760,8.0000000000,nan,0.0000000000,14,gps,0.8000000000,1.3000000000,1.5000000000,3.6000000000,nan,nan,nan,85,nan


In [16]:
raw_gps.tail()

,time,lat,lon,elevation,accuracy,bearing,speed,satellites,provider,hdop,vdop,pdop,geoidheight,ageofdgpsdata,dgpsid,activity,battery,annotation
1029,2019-12-26T22:38:24.000Z,-27.7177782705,-51.0991311244,977.4991457167,4.0000000000,137.6589400000,1.4786099000,21,gps,0.8000000000,1.3000000000,1.5000000000,3.6000000000,nan,nan,nan,82,nan
1030,2019-12-26T22:38:25.000Z,-27.7177886792,-51.0991222101,977.2160921851,4.0000000000,136.7660400000,1.0100127000,21,gps,0.8000000000,1.3000000000,1.5000000000,3.6000000000,nan,nan,nan,82,nan
1031,2019-12-26T22:38:26.000Z,-27.7177921860,-51.0991188802,977.3247762051,4.0000000000,133.6576400000,0.4891710600,21,gps,0.8000000000,1.3000000000,1.5000000000,3.6000000000,nan,nan,nan,82,nan
1032,2019-12-26T22:38:27.000Z,-27.7177915733,-51.0991188344,977.4790654537,4.0000000000,134.7767200000,0.1219917240,21,gps,0.8000000000,1.3000000000,1.5000000000,3.6000000000,nan,nan,nan,82,nan
1033,2019-12-26T22:38:28.000Z,-27.7177891462,-51.0991198606,977.7430051602,4.0000000000,134.8069600000,0.0115761785,21,gps,0.8000000000,1.3000000000,1.5000000000,3.6000000000,nan,nan,nan,82,nan


<h4>Processamento</h4>

In [17]:
# Remove dados de outros provedores, como network, pois introduzem outliers.
def removeOtherProviders(data):
    
    iniSize = len(data)
    data = data[data.provider == 'gps']
    data = data.reset_index(drop=True)
    endSize = len(data)
    
    print("Removido", (iniSize - endSize), "registros de outros provedores diferentes de GPS.")
    
    return data

# Calcula o tempo decorrido entre amostras
def calcElapsedTime(timestamps):
    
    elapsed = [0]
    size = len(timestamps)
    
    for i in range(1, size):
        elapsed.append(abs(timestamps[i] - timestamps[i-1]))
    
    return pd.Series(elapsed)

# Calcula a dostância entre os pontos geodésicos
def calcDistance(latitudes, longitudes):
    
    distance = [0]
    size = len(latitudes)
    
    for i in range(1, size):
        start = (latitudes[i-1], longitudes[i-1])
        end = (latitudes[i], longitudes[i])
        # dist = abs(haversine(start, end, unit='m'))
        dist = geodesic(start, end).meters
        distance.append(dist)
    
    return pd.Series(distance)

# Busca amostras de localização com tempo errado na sequência temporal
def findOutliers(timestamps):
    
    size = len(timestamps)
    outliers = []
    
    for i in range(0, size):
        
        if i > 0:
            before = timestamps[i-1]
        else:
            before = timestamps[i]
        
        current = timestamps[i] 
        
        if i < size - 1:
            after = timestamps[i+1]
        else:
            after = timestamps[i]
            
        if before > current or current > after:
            outliers.append(i)
    
    return outliers

# Método Principal que processa os dados do GPS
def processGPS(raw_gps):
    
    gps = removeOtherProviders(raw_gps)
    gps.insert(0, 'timestamp', gps['time'].apply(parseDateToTimestamp), True) 
    gps = gps.drop(columns=['time'])
    gps = gps.rename(columns={"lat": "latitude", "lon": "longitude", "speed": "speed_meters_per_second"})
    gps.insert(len(gps.columns), 'distance_meters', calcDistance(gps['latitude'], gps['longitude']), True)
    gps.insert(len(gps.columns), 'elapsed_time_seconds', calcElapsedTime(gps['timestamp']), True)
    
    outliers = findOutliers(gps['timestamp'])
    print("Quantidade de Outliers =", len(outliers))
    file = saveFolder + "dataset_gps.csv"
    gps.to_csv(file, index=False)
    print("Salvo em", file)
    return gps

# Método Que Cria um Mapa e Mostra as Localizações
def plotAndSaveMap(gps):
    
    locations = gps[['latitude', 'longitude']]
    coordinates = [tuple(x) for x in locations.to_numpy()]

    focolat = (gps['latitude'].min() + gps['latitude'].max()) / 2
    focolon = (gps['longitude'].min() + gps['longitude'].max()) / 2
    maps = folium.Map(location=[focolat, focolon], zoom_start=14)

    folium.PolyLine(coordinates, color="blue", weight=2.5, opacity=1).add_to(maps)

    maps.save(saveFolder + 'dataset_map.html')
    return maps

In [18]:
data_gps = processGPS(raw_gps.copy())
plotAndSaveMap(data_gps)

Removido 20 registros de outros provedores diferentes de GPS.
Quantidade de Outliers = 0
Salvo em E:\Dados Processados\100 Hz\2019-12-26 19-23-31 - Palio Cenário 3\dataset_gps.csv


<h2> Correção de Configurações do MPU </h2>

<h4>Visualização de Dados</h4>

In [19]:
settings_68_direita.head()

,timestamp,address_mpu_master,address_mpu_slave,address_ak,gyroscope_full_scale,accelerometer_full_scale,magnetometer_full_scale,gyroscope_resolution,accelerometer_resolution,magnetometer_resolution,gyroscope_master_bias_x,gyroscope_master_bias_y,gyroscope_master_bias_z,gyroscope_slave_bias_x,gyroscope_slave_bias_y,...,accelerometer_master_bias_x,accelerometer_master_bias_y,accelerometer_master_bias_z,accelerometer_slave_bias_x,accelerometer_slave_bias_y,accelerometer_slave_bias_z,magnetometer_factory_sensitivity_x,magnetometer_factory_sensitivity_y,magnetometer_factory_sensitivity_z,magnetometer_soft_iron_distortion_x,magnetometer_soft_iron_distortion_y,magnetometer_soft_iron_distortion_z,magnetometer_hard_iron_distortion_x,magnetometer_hard_iron_distortion_y,magnetometer_hard_iron_distortion_z
0,1577398825.9248919487,0x68,0x68,0xc,GFS_1000,AFS_8G,AK8963_BIT_16,0.0305175781,0.0002441406,0.1499389499,0.5576901320,-0.4532976848,-0.7413538491,-0.9034319622,-0.1161156631,...,-0.2396686833,-0.0421470084,-0.0548482755,-0.1255597370,0.1684868045,0.3139648438,1.1875000000,1.1953125000,1.1367187500,1.1538461538,1.0227272727,0.8653846154,14.2316077152,3.9429258242,4.6598214286


In [20]:
settings_69_direita.head()

,timestamp,address_mpu_master,address_mpu_slave,address_ak,gyroscope_full_scale,accelerometer_full_scale,magnetometer_full_scale,gyroscope_resolution,accelerometer_resolution,magnetometer_resolution,gyroscope_master_bias_x,gyroscope_master_bias_y,gyroscope_master_bias_z,gyroscope_slave_bias_x,gyroscope_slave_bias_y,...,accelerometer_master_bias_x,accelerometer_master_bias_y,accelerometer_master_bias_z,accelerometer_slave_bias_x,accelerometer_slave_bias_y,accelerometer_slave_bias_z,magnetometer_factory_sensitivity_x,magnetometer_factory_sensitivity_y,magnetometer_factory_sensitivity_z,magnetometer_soft_iron_distortion_x,magnetometer_soft_iron_distortion_y,magnetometer_soft_iron_distortion_z,magnetometer_hard_iron_distortion_x,magnetometer_hard_iron_distortion_y,magnetometer_hard_iron_distortion_z
0,1577398825.9290544987,0x69,nan,0xc,GFS_1000,AFS_8G,AK8963_BIT_16,0.0305175781,0.0002441406,0.1499389499,0.9283670565,0.5154493378,0.8157869665,0,0,...,-0.2060368236,0.1612161776,0.2777843941,0,0,0,1.1757812500,1.1757812500,1.1406250000,1.0000000000,1.1600000000,0.8787878788,36.0524105235,20.0095285791,-43.6966613248


In [21]:
settings_68_esquerda.head()

,timestamp,address_mpu_master,address_mpu_slave,address_ak,gyroscope_full_scale,accelerometer_full_scale,magnetometer_full_scale,gyroscope_resolution,accelerometer_resolution,magnetometer_resolution,gyroscope_master_bias_x,gyroscope_master_bias_y,gyroscope_master_bias_z,gyroscope_slave_bias_x,gyroscope_slave_bias_y,...,accelerometer_master_bias_x,accelerometer_master_bias_y,accelerometer_master_bias_z,accelerometer_slave_bias_x,accelerometer_slave_bias_y,accelerometer_slave_bias_z,magnetometer_factory_sensitivity_x,magnetometer_factory_sensitivity_y,magnetometer_factory_sensitivity_z,magnetometer_soft_iron_distortion_x,magnetometer_soft_iron_distortion_y,magnetometer_soft_iron_distortion_z,magnetometer_hard_iron_distortion_x,magnetometer_hard_iron_distortion_y,magnetometer_hard_iron_distortion_z
0,1577399000.1960000992,0x68,0x68,0xc,GFS_1000,AFS_8G,AK8963_BIT_16,0.0305175781,0.0002441406,0.1499389499,1.4661463296,-0.2826597632,1.2735506383,1.1781964983,0.3611246745,...,0.1028606136,0.0697825362,-0.0022746761,-0.0859781901,0.0717831566,-0.0467064267,1.1328125000,1.1875000000,1.1445312500,3.1604938272,2.6666666667,0.4331641286,-30.6323517628,24.4587912088,-59.9776299222


In [22]:
settings_69_esquerda.head()

,timestamp,address_mpu_master,address_mpu_slave,address_ak,gyroscope_full_scale,accelerometer_full_scale,magnetometer_full_scale,gyroscope_resolution,accelerometer_resolution,magnetometer_resolution,gyroscope_master_bias_x,gyroscope_master_bias_y,gyroscope_master_bias_z,gyroscope_slave_bias_x,gyroscope_slave_bias_y,...,accelerometer_master_bias_x,accelerometer_master_bias_y,accelerometer_master_bias_z,accelerometer_slave_bias_x,accelerometer_slave_bias_y,accelerometer_slave_bias_z,magnetometer_factory_sensitivity_x,magnetometer_factory_sensitivity_y,magnetometer_factory_sensitivity_z,magnetometer_soft_iron_distortion_x,magnetometer_soft_iron_distortion_y,magnetometer_soft_iron_distortion_z,magnetometer_hard_iron_distortion_x,magnetometer_hard_iron_distortion_y,magnetometer_hard_iron_distortion_z
0,1577399000.2004036903,0x69,nan,0xc,GFS_1000,AFS_8G,AK8963_BIT_16,0.0305175781,0.0002441406,0.1499389499,-0.4352569580,0.9391784668,0.5317687988,0,0,...,-0.1636840820,0.0762329102,0.4408081055,0,0,0,1.1914062500,1.1953125000,1.1484375000,1.0119047619,1.0493827160,0.9444444444,26.6170921093,18.9081215659,-50.7976762821


<h4>Processamento</h4>

In [23]:
# Converte de g para m/s^2
def convertAccelerationSettings(data):
    
    data['accelerometer_bias_x'] *= gravity
    data['accelerometer_bias_y'] *= gravity
    data['accelerometer_bias_z'] *= gravity
    
    return data

# Método Principal que processa as configurações dos MPUs
def processSettings(data_68, data_69, side):
    
    data = pd.DataFrame(columns = [
        'placement', 
        'address_mpu', 
        'address_ak',
        'gyroscope_full_scale', 
        'accelerometer_full_scale',
        'magnetometer_full_scale', 
        'gyroscope_resolution',
        'accelerometer_resolution', 
        'magnetometer_resolution',
        'gyroscope_bias_x', 
        'gyroscope_bias_y',
        'gyroscope_bias_z',
        'accelerometer_bias_x', 
        'accelerometer_bias_y',
        'accelerometer_bias_z',
        'magnetometer_factory_sensitivity_x',
        'magnetometer_factory_sensitivity_y',
        'magnetometer_factory_sensitivity_z',
        'magnetometer_soft_iron_distortion_x',
        'magnetometer_soft_iron_distortion_y',
        'magnetometer_soft_iron_distortion_z',
        'magnetometer_hard_iron_distortion_x',
        'magnetometer_hard_iron_distortion_y',
        'magnetometer_hard_iron_distortion_z'
    ])
    
    data = data.append({
        'placement': 'dashboard',
        'address_mpu': data_69['address_mpu_master'].values[0],
        'address_ak': data_69['address_ak'].values[0],
        'gyroscope_full_scale': data_69['gyroscope_full_scale'].values[0],
        'accelerometer_full_scale': data_69['accelerometer_full_scale'].values[0],
        'magnetometer_full_scale': data_69['magnetometer_full_scale'].values[0],
        'gyroscope_resolution': data_69['gyroscope_resolution'].values[0],
        'accelerometer_resolution': data_69['accelerometer_resolution'].values[0],
        'magnetometer_resolution': data_69['magnetometer_resolution'].values[0],
        'gyroscope_bias_x': data_69['gyroscope_master_bias_x'].values[0],
        'gyroscope_bias_y': data_69['gyroscope_master_bias_y'].values[0],
        'gyroscope_bias_z': data_69['gyroscope_master_bias_z'].values[0],
        'accelerometer_bias_x': data_69['accelerometer_master_bias_x'].values[0],
        'accelerometer_bias_y': data_69['accelerometer_master_bias_y'].values[0],
        'accelerometer_bias_z': data_69['accelerometer_master_bias_z'].values[0],
        'magnetometer_factory_sensitivity_x': data_69['magnetometer_factory_sensitivity_x'].values[0],
        'magnetometer_factory_sensitivity_y': data_69['magnetometer_factory_sensitivity_y'].values[0],
        'magnetometer_factory_sensitivity_z': data_69['magnetometer_factory_sensitivity_z'].values[0],
        'magnetometer_soft_iron_distortion_x': data_69['magnetometer_soft_iron_distortion_x'].values[0],
        'magnetometer_soft_iron_distortion_y': data_69['magnetometer_soft_iron_distortion_y'].values[0],
        'magnetometer_soft_iron_distortion_z': data_69['magnetometer_soft_iron_distortion_z'].values[0],
        'magnetometer_hard_iron_distortion_x': data_69['magnetometer_hard_iron_distortion_x'].values[0],
        'magnetometer_hard_iron_distortion_y': data_69['magnetometer_hard_iron_distortion_y'].values[0],
        'magnetometer_hard_iron_distortion_z': data_69['magnetometer_hard_iron_distortion_z'].values[0],
    }, ignore_index=True)
    
    data = data.append({
        'placement': 'above_suspension',
        'address_mpu': data_68['address_mpu_master'].values[0],
        'address_ak': data_68['address_ak'].values[0],
        'gyroscope_full_scale': data_68['gyroscope_full_scale'].values[0],
        'accelerometer_full_scale': data_68['accelerometer_full_scale'].values[0],
        'magnetometer_full_scale': data_68['magnetometer_full_scale'].values[0],
        'gyroscope_resolution': data_68['gyroscope_resolution'].values[0],
        'accelerometer_resolution': data_68['accelerometer_resolution'].values[0],
        'magnetometer_resolution': data_68['magnetometer_resolution'].values[0],
        'gyroscope_bias_x': data_68['gyroscope_master_bias_x'].values[0],
        'gyroscope_bias_y': data_68['gyroscope_master_bias_y'].values[0],
        'gyroscope_bias_z': data_68['gyroscope_master_bias_z'].values[0],
        'accelerometer_bias_x': data_68['accelerometer_master_bias_x'].values[0],
        'accelerometer_bias_y': data_68['accelerometer_master_bias_y'].values[0],
        'accelerometer_bias_z': data_68['accelerometer_master_bias_z'].values[0],
        'magnetometer_factory_sensitivity_x': data_68['magnetometer_factory_sensitivity_x'].values[0],
        'magnetometer_factory_sensitivity_y': data_68['magnetometer_factory_sensitivity_y'].values[0],
        'magnetometer_factory_sensitivity_z': data_68['magnetometer_factory_sensitivity_z'].values[0],
        'magnetometer_soft_iron_distortion_x': data_68['magnetometer_soft_iron_distortion_x'].values[0],
        'magnetometer_soft_iron_distortion_y': data_68['magnetometer_soft_iron_distortion_y'].values[0],
        'magnetometer_soft_iron_distortion_z': data_68['magnetometer_soft_iron_distortion_z'].values[0],
        'magnetometer_hard_iron_distortion_x': data_68['magnetometer_hard_iron_distortion_x'].values[0],
        'magnetometer_hard_iron_distortion_y': data_68['magnetometer_hard_iron_distortion_y'].values[0],
        'magnetometer_hard_iron_distortion_z': data_68['magnetometer_hard_iron_distortion_z'].values[0],
    }, ignore_index=True)
    
    data = data.append({
        'placement': 'below_suspension',
        'address_mpu': data_68['address_mpu_slave'].values[0],
        'gyroscope_full_scale': data_68['gyroscope_full_scale'].values[0],
        'accelerometer_full_scale': data_68['accelerometer_full_scale'].values[0],
        'magnetometer_full_scale': data_68['magnetometer_full_scale'].values[0],
        'gyroscope_resolution': data_68['gyroscope_resolution'].values[0],
        'accelerometer_resolution': data_68['accelerometer_resolution'].values[0],
        'magnetometer_resolution': data_68['magnetometer_resolution'].values[0],
        'gyroscope_bias_x': data_68['gyroscope_slave_bias_x'].values[0],
        'gyroscope_bias_y': data_68['gyroscope_slave_bias_y'].values[0],
        'gyroscope_bias_z': data_68['gyroscope_slave_bias_z'].values[0],
        'accelerometer_bias_x': data_68['accelerometer_slave_bias_x'].values[0],
        'accelerometer_bias_y': data_68['accelerometer_slave_bias_y'].values[0],
        'accelerometer_bias_z': data_68['accelerometer_slave_bias_z'].values[0]
    }, ignore_index=True)
    
    # Conversão da aceleração
    data = convertAccelerationSettings(data)
    
    data = data.reset_index(drop=True)
    
    file = saveFolder + "dataset_settings_" + translateSide(side) + ".csv"
    data.to_csv(file, index=False)
    print("Salvo em", file)
    
    return data

In [24]:
settings_direita = processSettings(settings_68_direita.copy(), settings_69_direita.copy(), 'direita')

Salvo em E:\Dados Processados\100 Hz\2019-12-26 19-23-31 - Palio Cenário 3\dataset_settings_right.csv


In [25]:
settings_esquerda = processSettings(settings_68_esquerda.copy(), settings_69_esquerda.copy(), 'esquerda')

Salvo em E:\Dados Processados\100 Hz\2019-12-26 19-23-31 - Palio Cenário 3\dataset_settings_left.csv


<h2> Correção de Dados do MPU </h2>

<h4>Visualização de Dados</h4>

In [26]:
raw_data_68_direita.describe()

,timestamp,master_acc_x,master_acc_y,master_acc_z,master_gyro_x,master_gyro_y,master_gyro_z,slave_acc_x,slave_acc_y,slave_acc_z,slave_gyro_x,slave_gyro_y,slave_gyro_z,mag_x,mag_y,mag_z,master_temp,slave_temp
count,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000,140347.0000000000
mean,1577399398.1608743668,0.0428730325,-0.0243903150,0.9991930604,-0.0267027118,0.1240430524,-0.1907146582,0.0397306284,-0.0314788891,1.0084489370,-0.1016595928,-0.1097542507,-0.2238414314,-2.1786573123,-8.9332514158,-9.0501779336,34.7469366106,37.2061030411
std,325.9780961850,0.1881675806,0.2064167393,0.2797207660,4.9808962483,7.1895115859,4.2795496547,0.3103258976,0.6390793808,0.3489513419,7.9083743869,18.5570028096,7.9583462783,9.0333164197,21.1465093439,15.0721320569,2.9038376705,2.5315812990
min,1577398831.8611865044,-1.6013957698,-1.6849037729,-1.2137064120,-131.7222409132,-225.8040265339,-38.2601109947,-3.0243426067,-6.7619926639,-2.7988281250,-147.6256207722,-309.6067847275,-91.0090004526,-16.4210858252,-50.9566136988,-72.4701170224,30.4887231557,20.9970048222
25%,1577399117.9317293167,-0.0186320979,-0.1001869760,0.9007955412,-1.6563229445,-2.3238019245,-1.3948766197,-0.0393572551,-0.1672661014,0.9042968750,-2.5145367878,-4.4004858994,-2.5690590463,-9.4359491670,-26.7613870504,-17.8970473637,32.9807110552,34.8017791356
50%,1577399399.7939398289,0.0236042302,-0.0071693979,0.9957662443,-0.0388913038,0.0870867473,-0.0521031822,0.0134991902,-0.0005180545,1.0017089844,-0.1036481159,-0.0364722275,-0.0361000619,-2.8617029005,-2.5661604021,-2.8525795118,33.6995537185,36.5270015275
75%,1577399679.2562153339,0.1285846989,0.0521567740,1.0980611662,1.4564700243,2.8336687786,1.2601526772,0.1452130574,0.1051948361,1.1132812500,2.6124163372,5.4566918350,2.0696128287,1.8635366036,2.5661604021,0.8347900597,34.8017791356,39.8816006230
max,1577399961.8644649982,1.3402546208,1.6390708365,4.4371724943,214.5607180712,179.7745867473,24.9112757241,3.7273663777,6.8862006955,5.2199707031,82.9957171184,232.5685082412,72.0159018912,33.5020967614,34.6431654283,32.6936631578,40.7921346632,42.4215113667


In [27]:
raw_data_69_direita.describe()

,timestamp,master_acc_x,master_acc_y,master_acc_z,master_gyro_x,master_gyro_y,master_gyro_z,slave_acc_x,slave_acc_y,slave_acc_z,slave_gyro_x,slave_gyro_y,slave_gyro_z,mag_x,mag_y,mag_z,master_temp,slave_temp
count,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000,139897.0000000000
mean,1577399397.4515058994,0.0380267398,-0.0233657251,1.0061593725,0.0297357303,0.0473326114,-0.1349459218,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-10.2140004865,-7.2565513631,-0.5544465866,28.0187932545,0.0000000000
std,326.2004154433,0.1240537322,0.1911115558,0.2093283241,6.6670564310,6.2548975969,4.2119089641,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,12.8198018604,9.3267386286,1.9274578109,0.9593684973,0.0000000000
min,1577398831.8602201939,-0.8820979421,-1.7669290682,-0.8095226753,-67.3040994784,-47.2073438691,-38.5049959508,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-35.1709334936,-28.3236199252,-12.2489544021,26.4632042412,0.0000000000
25%,1577399117.2784588337,-0.0063655202,-0.0823587557,0.9353503716,-1.8744119784,-2.1939161347,-1.1819979040,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-26.0035723825,-15.2354489850,-1.8786739880,27.0862012160,0.0000000000
50%,1577399397.9670557976,0.0261051829,-0.0025247713,1.0007800591,-0.0128397127,0.0338670684,0.0081876429,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-4.4955328526,-5.6238234509,-0.3757347976,27.8529667236,0.0000000000
75%,1577399680.4024207592,0.0991032298,0.0423971037,1.0803699028,2.0013204435,2.1700975372,1.1678556117,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.6170339209,0.3067540064,0.6763226357,28.8114236080,0.0000000000
max,1577399961.8721449375,0.8867008861,1.2809224943,3.0740222466,104.3572774748,72.6351854278,24.3917325648,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,10.6658720620,10.5318875534,7.1389611546,30.2491089346,0.0000000000


In [28]:
raw_data_68_esquerda.describe()

,timestamp,master_acc_x,master_acc_y,master_acc_z,master_gyro_x,master_gyro_y,master_gyro_z,slave_acc_x,slave_acc_y,slave_acc_z,slave_gyro_x,slave_gyro_y,slave_gyro_z,mag_x,mag_y,mag_z,master_temp,slave_temp
count,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000,114056.0000000000
mean,1577399462.7419958115,0.0569665494,-0.0230614666,0.9864320260,-0.1274982227,0.0431570678,0.0758680119,0.0451287771,-0.0239329298,1.0005112358,0.0339264628,-0.1636953390,-0.1300687801,-46.3797250479,-27.6634595387,3.2391995721,34.3061392098,32.6936503761
std,263.9003050831,0.2511223544,0.2579044472,0.3581527704,5.2099566400,6.8503402798,4.7652161720,0.2584587053,0.7513252149,0.4176105787,3.6311391039,23.5648047939,8.4615087675,44.9326340835,35.5667524213,7.1802772902,2.4919544213,1.9863839045
min,1577399005.3975059986,-1.7229778011,-1.8537180831,-1.5275104802,-124.2078455483,-226.5544984399,-38.5660311071,-2.1325276693,-6.6474667504,-4.7164771670,-29.5595441546,-233.9731852214,-71.0801624116,-178.0376942673,-308.3244607245,-23.8987717678,0.0000000000,0.0000000000
25%,1577399234.1424276829,-0.0564738948,-0.1410715987,0.8208781917,-2.5952967202,-3.3489320336,-1.7923494665,-0.0573323568,-0.2353573754,0.8301536923,-1.7275129046,-8.8755289714,-3.3616565523,-101.2726540949,-58.1013431013,-3.3822523031,33.0286338994,30.8721059095
50%,1577399462.9095597267,0.0485065739,-0.0155833175,0.9908000667,-0.1233728921,0.0385191382,0.2828458460,0.0400797526,-0.0122128441,1.0010521298,-0.0185285296,-0.6357828776,0.0257946196,-32.0230724009,-34.3610093610,7.0246778602,33.8912450954,32.4535597688
75%,1577399691.1947948933,0.1786335271,0.1033131669,1.1507121761,2.2264806236,3.3954527320,2.2359708460,0.1600138346,0.2023867653,1.1724388486,1.7514910017,7.3293050130,3.2606579009,-6.2557861891,-0.1749287749,8.1397060920,34.9455476683,34.5621649145
max,1577399920.9387981892,1.7335651677,1.8088795732,3.5977336604,248.6254064048,124.8554136695,25.4293302210,1.9875895182,6.5739687965,5.9143822080,28.6985124860,372.8688557943,61.7933727446,106.4760909872,73.4201058201,21.4457096579,42.5173570551,37.1979213466


In [29]:
raw_data_69_esquerda.describe()

,timestamp,master_acc_x,master_acc_y,master_acc_z,master_gyro_x,master_gyro_y,master_gyro_z,slave_acc_x,slave_acc_y,slave_acc_z,slave_gyro_x,slave_gyro_y,slave_gyro_z,mag_x,mag_y,mag_z,master_temp,slave_temp
count,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000,114124.0000000000
mean,1577399462.7576119900,0.0468647592,-0.0213933934,1.0048731493,-0.0096237107,0.0221679090,-0.1677760441,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-11.3109115938,-7.4830858291,-0.9373987574,29.9201769153,0.0000000000
std,263.8681290861,0.1332662197,0.2094187812,0.2479413359,5.1061938885,4.9748861572,4.5852281937,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,13.1817790109,8.7982942713,1.8467618883,0.6525150569,0.0000000000
min,1577399005.4013316631,-0.7999389648,-1.5012817383,-1.1488159180,-37.7117156982,-57.1220397949,-38.1599426270,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-34.7068506890,-25.4840898877,-13.6608440171,0.0000000000,0.0000000000
25%,1577399234.2202382088,-0.0184448242,-0.1065063477,0.8975708008,-1.8230438232,-1.7631530762,-1.7524719238,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-26.9339622534,-14.7638454331,-2.2768073362,29.3385748944,0.0000000000
50%,1577399462.8374369144,0.0418579102,-0.0132446289,1.0040161133,0.0080108643,0.0373840332,0.0175476074,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-6.6882993515,-7.2408668685,-0.8131454772,29.8657261808,0.0000000000
75%,1577399691.1778512001,0.1204711914,0.0731811523,1.1153442383,1.9611358643,1.9599914551,1.6960144043,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.2653539314,-0.2821116962,0.3252581909,30.3449546231,0.0000000000
max,1577399920.9461736679,0.9315063477,1.3004760742,3.2149536133,41.0236358643,55.7624816895,24.2179870605,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,10.4843611456,11.1904306149,7.6435674858,31.6388714170,0.0000000000


In [30]:
raw_data_68_direita.head()

,timestamp,master_acc_x,master_acc_y,master_acc_z,master_gyro_x,master_gyro_y,master_gyro_z,slave_acc_x,slave_acc_y,slave_acc_z,slave_gyro_x,slave_gyro_y,slave_gyro_z,mag_x,mag_y,mag_z,master_temp,slave_temp
0,1577398831.8611865044,-0.0039836604,-0.0076576791,0.9842916349,0.0221438524,0.0260515911,-0.1131383384,0.0034894245,0.0021674924,1.0004882812,-0.1036481159,-0.1890601181,-0.0971352182,-1.6290317255,1.8329717158,0.6872952768,39.6419864019,41.6547458592
1,1577398831.8692429066,0.0006550114,-0.0066811166,0.9916158537,-0.2525143507,-0.0349835652,0.0089319741,0.0005597370,0.0011909299,0.9943847656,0.0489397747,0.0245629287,-0.0666176401,-1.6290317255,1.8329717158,0.6872952768,39.5461407135,40.9838260401
2,1577398831.8771333694,-0.0034953792,-0.0035072885,0.9938131193,0.1747317430,-0.0044659870,0.0699671303,0.0025128620,-0.0022270389,0.9963378906,-0.0426129597,-0.1280249619,-0.2192055307,-1.6290317255,-4.0325377747,0.6872952768,39.6419864019,41.6547458592
3,1577398831.8849833012,0.0021198552,-0.0025307260,0.9881978849,0.2357668993,0.2396746380,-0.2046910728,0.0005597370,0.0036323361,1.0012207031,-0.2867535847,0.0245629287,-0.1581703744,-1.6290317255,-2.5661604021,0.6872952768,39.9774463114,41.7985143918
4,1577398831.8928644657,-0.0005656917,-0.0103432260,0.9945455412,-0.0083737257,0.1176043255,0.1004847085,0.0037335652,0.0009467893,1.0014648438,-0.1036481159,-0.2195776963,-0.0361000619,-1.6290317255,-2.5661604021,0.6872952768,39.5461407135,41.6547458592


In [31]:
raw_data_69_direita.head()

,timestamp,master_acc_x,master_acc_y,master_acc_z,master_gyro_x,master_gyro_y,master_gyro_z,slave_acc_x,slave_acc_y,slave_acc_z,slave_gyro_x,slave_gyro_y,slave_gyro_z,mag_x,mag_y,mag_z,master_temp,slave_temp
0,1577398831.8602201939,-0.0022151296,0.0013814787,0.9988269341,-0.0128397127,0.0643846465,-0.0223299352,0,0,0,0,0,0,1.3222155449,0.5112566774,0.5260287167,28.2842723216,0
1,1577398831.8681623936,-0.0031916921,-0.0000833651,0.9963855278,-0.1959451815,0.0643846465,0.0387052210,0,0,0,0,0,0,0.0881477030,-0.5112566774,0.3757347976,28.4759636984,0
2,1577398831.8759775162,0.0009586986,-0.0032571932,0.9944324028,0.1092305998,-0.0882032441,-0.1749178258,0,0,0,0,0,0,0.0881477030,-0.5112566774,0.3757347976,28.4280408542,0
3,1577398831.8837816715,-0.0014827077,-0.0022806307,0.9963855278,-0.1654276034,0.1864549590,-0.1444002477,0,0,0,0,0,0,0.0881477030,-0.5112566774,0.3757347976,28.2842723216,0
4,1577398831.8916490078,-0.0002620046,0.0011373380,0.9946765434,0.0787130216,-0.0271680878,0.1302579554,0,0,0,0,0,0,0.0881477030,1.1247646902,0.0751469595,28.4759636984,0


In [32]:
raw_data_68_esquerda.head()

,timestamp,master_acc_x,master_acc_y,master_acc_z,master_gyro_x,master_gyro_y,master_gyro_z,slave_acc_x,slave_acc_y,slave_acc_z,slave_gyro_x,slave_gyro_y,slave_gyro_z,mag_x,mag_y,mag_z,master_temp,slave_temp
0,1577399005.3975059986,0.0091999333,0.0066334794,1.0037395198,-0.2454432046,0.0080015601,0.1302579554,0.0000406901,-0.0000058129,0.9932396298,-0.0490461077,-0.2695719401,-0.0962756929,-1.9612384872,3.1487179487,-5.8353144130,42.0860514572,36.7186929044
1,1577399005.4028644562,0.0057819646,0.0044362138,1.0076457698,-0.0318201577,0.0995542945,0.1607755335,0.0051676432,0.0014590309,0.9986107236,-0.1711164202,-0.2085367839,0.0257946196,3.4069461403,3.1487179487,-5.8353144130,42.1339743014,36.7186929044
2,1577399005.4105281830,0.0018757146,0.0022389482,1.0042278011,-0.1233728921,-0.0225160180,0.4659513148,0.0019938151,-0.0017147972,0.9973900205,0.1645769392,-0.0864664714,-0.2488635835,-4.6453308009,-3.9733821734,-5.6123087666,42.0860514572,36.6707700602
3,1577399005.4183940887,0.0106647771,0.0080983232,0.9993449886,-0.2759607827,-0.0835511743,0.3743985804,0.0041910807,0.0009707496,0.9998314267,0.1035417829,-0.2390543620,-0.1267932710,3.4069461403,-4.4481888482,-6.1326552748,41.9902057687,36.8624614371
4,1577399005.4262611866,0.0062702458,0.0039479325,0.9966594417,0.0597325767,0.0385191382,0.2828458460,0.0054117839,0.0012148903,0.9995872861,-0.1711164202,-0.0864664714,-0.1267932710,1.2596722893,-4.4481888482,-6.1326552748,42.1339743014,36.8624614371


In [33]:
raw_data_69_esquerda.head()

,timestamp,master_acc_x,master_acc_y,master_acc_z,master_gyro_x,master_gyro_y,master_gyro_z,slave_acc_x,slave_acc_y,slave_acc_z,slave_gyro_x,slave_gyro_y,slave_gyro_z,mag_x,mag_y,mag_z,master_temp,slave_temp
0,1577399005.4013316631,-0.0025756836,0.0060424805,1.0020629883,0.0385284424,0.2510070801,-0.0434875488,0,0,0,0,0,0,0.9038242367,0.0940372321,-0.8131454772,31.0158744421,0
1,1577399005.4093515873,-0.0015991211,0.0057983398,1.0032836914,-0.0530242920,-0.0846862793,0.0480651855,0,0,0,0,0,0,-0.3615296947,-0.8463350885,-0.6505163818,31.0637972864,0
2,1577399005.4171948433,-0.0042846680,0.0036010742,1.0003540039,0.0080108643,0.1289367676,0.1396179199,0,0,0,0,0,0,-1.6268836260,0.4701861603,0.1626290954,30.9200287537,0
3,1577399005.4250483513,-0.0040405273,0.0048217773,0.9976684570,-0.0530242920,-0.0541687012,-0.0434875488,0,0,0,0,0,0,0.0000000000,-1.5986329450,-0.6505163818,30.8721059095,0
4,1577399005.4331378937,0.0020629883,0.0067749023,1.0008422852,0.2216339111,-0.0541687012,-0.0740051270,0,0,0,0,0,0,0.0000000000,-1.5986329450,-0.6505163818,30.8721059095,0


<h4>Processamento</h4>

In [34]:
# Corrige Arredondamento e Duplicação de Timestamps
def fixTimestamps(data):
    
    # Arredonda Timestamps
    data['timestamp'] = data['timestamp'].apply(roundTimestamp)
    
    # Agrupa Timestamps - Média
    data = data.groupby(["timestamp"], as_index=False).mean()
    
    data = data.reset_index(drop=True)
    
    return data

# Sequencia as amostras do inicio ao fim, em um intervalo fixo de passo
def sequencing(data):

    index = -1
    series = None
    timestamp = 0
    ini = iniTime
    end = endTime
    new_points = []
    result = []
    
    while(timestamp < ini):
        index += 1
        series = data.iloc[index].copy()
        timestamp = roundTimestamp(series['timestamp'])

    while ini <= end and index < len(data):
        
        series = data.iloc[index].copy()
        timestamp = roundTimestamp(series['timestamp'])
            
        if ini == timestamp:
            index += 1 

        else:
            series = (result[-1].copy() + series) / 2.0
            new_points.append(len(result))
        
        series['timestamp'] = ini
        result.append(series)
        ini = roundTimestamp(ini + sampling_rate_interval)
        
        load_bar.update(1)

    dataFrame = pd.DataFrame(result)
    dataFrame = dataFrame.reset_index(drop=True)
    return new_points, dataFrame

# Une os dados do 0x68 e 0x69 em um arquivo com novos labels de colunas
def mergeData(data_68, data_69):
    
    data_68_global = data_68
    data_69_global = data_69
    
    # Alteração de nomes de colunas

    data_68 = data_68.rename(columns={
        'master_acc_x': 'acc_x_above_suspension',
        'master_acc_y': 'acc_y_above_suspension',
        'master_acc_z': 'acc_z_above_suspension',
        'master_gyro_x': 'gyro_x_above_suspension',
        'master_gyro_y': 'gyro_y_above_suspension',
        'master_gyro_z': 'gyro_z_above_suspension',
        'slave_acc_x': 'acc_x_below_suspension',
        'slave_acc_y': 'acc_y_below_suspension',
        'slave_acc_z': 'acc_z_below_suspension',
        'slave_gyro_x': 'gyro_x_below_suspension',
        'slave_gyro_y': 'gyro_y_below_suspension',
        'slave_gyro_z': 'gyro_z_below_suspension',
        'mag_x': 'mag_x_above_suspension',
        'mag_y': 'mag_y_above_suspension',
        'mag_z': 'mag_z_above_suspension',
        'master_temp': 'temp_above_suspension',
        'slave_temp': 'temp_below_suspension'
    })

    data_69 = data_69.rename(columns={
        'master_acc_x': 'acc_x_dashboard',
        'master_acc_y': 'acc_y_dashboard',
        'master_acc_z': 'acc_z_dashboard',
        'master_gyro_x': 'gyro_x_dashboard',
        'master_gyro_y': 'gyro_y_dashboard',
        'master_gyro_z': 'gyro_z_dashboard',
        'mag_x': 'mag_x_dashboard',
        'mag_y': 'mag_y_dashboard',
        'mag_z': 'mag_z_dashboard',
        'master_temp': 'temp_dashboard'
    })
    
    # Exclusão de colunas não utilizadas
    
    data_69 = data_69.drop(columns=[
        'slave_acc_x', 
        'slave_acc_y', 
        'slave_acc_z', 
        'slave_gyro_x', 
        'slave_gyro_y', 
        'slave_gyro_z',
        'slave_temp'
    ])
    
    # Merge 0x68 e 0x69 para novo dataframe

    fieldsSource = {
        'timestamp': data_68,

        'acc_x_dashboard': data_69,
        'acc_y_dashboard': data_69,
        'acc_z_dashboard': data_69,
        
        'acc_x_above_suspension': data_68,
        'acc_y_above_suspension': data_68,
        'acc_z_above_suspension': data_68,
        
        'acc_x_below_suspension': data_68,
        'acc_y_below_suspension': data_68,
        'acc_z_below_suspension': data_68,

        'gyro_x_dashboard': data_69,
        'gyro_y_dashboard': data_69,
        'gyro_z_dashboard': data_69,
        
        'gyro_x_above_suspension': data_68,
        'gyro_y_above_suspension': data_68,
        'gyro_z_above_suspension': data_68,
        
        'gyro_x_below_suspension': data_68,
        'gyro_y_below_suspension': data_68,
        'gyro_z_below_suspension': data_68,

        'mag_x_dashboard': data_69,
        'mag_y_dashboard': data_69,
        'mag_z_dashboard': data_69,
        
        'mag_x_above_suspension': data_68,
        'mag_y_above_suspension': data_68,
        'mag_z_above_suspension': data_68,
        
        'temp_dashboard': data_69,
        'temp_above_suspension': data_68,
        'temp_below_suspension': data_68
    }

    index = 0
    data = pd.DataFrame()
    
    for key in fieldsSource:
        dataSource = fieldsSource[key]
        column = dataSource[key]        
        data.insert(index, column.name, column, True)
        index += 1

    return data

# Converte de g para m/s^2
def convertAcceleration(data):
        
    data['acc_x_dashboard'] *= gravity
    data['acc_y_dashboard'] *= gravity
    data['acc_z_dashboard'] *= gravity
    
    data['acc_x_above_suspension'] *= gravity
    data['acc_y_above_suspension'] *= gravity
    data['acc_z_above_suspension'] *= gravity
    
    data['acc_x_below_suspension'] *= gravity
    data['acc_y_below_suspension'] *= gravity
    data['acc_z_below_suspension'] *= gravity
    
    return data
    
# Método principal que processa os dados do MPU
def processMPU(data_68, data_69, side):
    
    print("Processando dados MPU do lado da", side)
    print("Ini", iniTime)
    print("End", endTime)
    print("Step", sampling_rate_interval)
    print("Samples", round((endTime - iniTime) / sampling_rate_interval) + 1)
    
    # Corrigindo Timestamps
    data_68 = fixTimestamps(data_68)
    data_69 = fixTimestamps(data_69)
    
    # Emparelhamento dos dados 0x68 e 0x69
    points_68, data_68 = sequencing(data_68)
    points_69, data_69 = sequencing(data_69)
    
    # Juntando os dados 0x68 e 0x69 
    data = mergeData(data_68, data_69)
    
    # Converte Aceleração de g para m/s^2
    data = convertAcceleration(data)

    # Salvando
    file = saveFolder + 'dataset_mpu_' + translateSide(side) + '.csv'
    data.to_csv(file, index=False)
    print("Salvo em ", file)
    
    print("Length Data 0x68 =", len(data_68))
    print("Length Data 0x69 =", len(data_69))
    print("\n", "Length New Points 0x68 =", len(points_68))
    print(points_68)
    print("\n","Length New Points 0x69 =", len(points_69))
    print(points_69)

    return data

In [35]:
count_progress = (round((endTime - iniTime) / sampling_rate_interval) * 2)

In [36]:
load_bar = tqdm_notebook(total=count_progress)
data_direita = processMPU(raw_data_68_direita.copy(), raw_data_69_direita.copy(), 'direita')

Processando dados MPU do lado da direita
Ini 1577399005.4
End 1577399920.94
Step 0.01
Samples 91555
Salvo em  E:\Dados Processados\100 Hz\2019-12-26 19-23-31 - Palio Cenário 3\dataset_mpu_right.csv
Length Data 0x68 = 91555
Length Data 0x69 = 91555

 Length New Points 0x68 = 1283
[389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 889, 890, 891, 1402, 1403, 2417, 2418, 2419, 2420, 2421, 2422, 2423, 2424, 2425, 2426, 2427, 2428, 2429, 2430, 2431, 2432, 2433, 2434, 2435, 2436, 2437, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 2445, 2446, 2447, 2448, 2449, 2450, 2451, 2452, 2453, 2454, 2455, 2456, 2457, 2458, 2459, 2460, 2461, 2462, 2463, 2464, 2465, 2466, 2467, 2468, 


 Length New Points 0x69 = 1516
[386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 1403, 1903, 1904, 1905, 1906, 2410, 2411, 2412, 2413, 2414, 2415, 2416, 2417, 2418, 2419, 2420, 2421, 2422, 2423, 2424, 2425, 2426, 2427, 2428, 2429, 2430, 2431, 2432, 2433, 2434, 2435, 2436, 2437, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 2445, 2446, 2447, 2448, 2449, 2450, 2451, 2452, 2453, 2454, 2455, 2456, 2457, 2458, 2459, 2460, 2461, 2462, 2463, 2464, 2465, 2466, 2467, 2468, 2469, 2470, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 2479, 2480, 2481, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489, 2490, 2491, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 25

In [37]:
load_bar = tqdm_notebook(total=count_progress)
data_esquerda = processMPU(raw_data_68_esquerda.copy(), raw_data_69_esquerda.copy(), 'esquerda')

Processando dados MPU do lado da esquerda
Ini 1577399005.4
End 1577399920.94
Step 0.01
Samples 91555
Salvo em  E:\Dados Processados\100 Hz\2019-12-26 19-23-31 - Palio Cenário 3\dataset_mpu_left.csv
Length Data 0x68 = 91555
Length Data 0x69 = 91555

 Length New Points 0x68 = 424
[184, 3518, 4106, 4107, 4108, 4109, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4133, 4134, 4135, 4136, 4137, 4138, 4139, 4140, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4153, 4154, 4155, 4156, 4157, 4158, 4159, 4160, 4161, 4162, 4163, 4164, 4165, 4166, 4167, 4168, 4169, 4170, 4171, 4172, 4173, 4174, 4175, 4176, 4177, 4178, 4179, 4180, 4181, 4182, 4183, 4184, 4185, 4186, 4187, 4188, 4189, 4190, 4191, 4192, 4193, 4194, 4195, 4196, 4197, 4198, 4199, 4200, 4201, 4202, 4203, 4204, 4205, 4206, 4207, 4208, 4209, 4210, 4211, 4212, 4213, 4214, 4215, 4216, 4217, 4218, 4219, 4220, 4221, 4222, 4223, 4

<h2> Correlação entre MPU e GPS </h2>

<h4>Visualização de Dados</h4>

In [38]:
data_direita.head()

,timestamp,acc_x_dashboard,acc_y_dashboard,acc_z_dashboard,acc_x_above_suspension,acc_y_above_suspension,acc_z_above_suspension,acc_x_below_suspension,acc_y_below_suspension,acc_z_below_suspension,gyro_x_dashboard,gyro_y_dashboard,gyro_z_dashboard,gyro_x_above_suspension,gyro_y_above_suspension,gyro_z_above_suspension,gyro_x_below_suspension,gyro_y_below_suspension,gyro_z_below_suspension,mag_x_dashboard,mag_y_dashboard,mag_z_dashboard,mag_x_above_suspension,mag_y_above_suspension,mag_z_above_suspension,temp_dashboard,temp_above_suspension,temp_below_suspension
0,1577399005.4000000954,-0.0720012353,0.0087592744,9.7472621198,-0.0270953554,-0.0367889523,9.7124586030,-0.0088760647,0.0428036541,9.7707369751,-0.0433572909,0.0033494903,0.0387052210,0.3273196337,0.1481219036,-0.1131383384,-0.0426129597,-0.2195776963,-0.1581703744,0.4407385150,-0.1022513355,0.0751469595,-1.6290317255,1.8329717158,0.9822848426,29.5302662713,40.2170605325,39.9774463114
1,1577399005.4100000858,-0.0432708154,0.0614317109,9.7544447248,-0.0127301454,-0.0367889523,9.7579484345,0.0342195652,0.0140732341,9.8593224365,-0.1043924471,0.0033494903,-0.1138826696,0.0526614305,-0.0349835652,0.0394495522,-0.1341656941,0.0855980850,-0.0666176401,-0.9696247329,1.5337700321,0.6763226357,-1.6290317255,-1.0997830295,0.9822848426,29.3385748944,40.2649833768,40.1691376883
2,1577399005.4200000763,-0.0049635888,0.0566433076,9.8071171613,-0.0247011537,-0.0152411374,9.7459774262,0.0270369602,0.0260442424,9.7755253784,0.1397481779,-0.0271680878,0.0692227992,-0.0999264601,-0.0349835652,-0.4183141197,-0.0426129597,0.0855980850,-0.0971352182,-0.9696247329,1.5337700321,0.6763226357,-1.4235865297,-3.8492406031,0.8347900597,29.5302662713,40.4087519094,40.0732919999
3,1577399005.4300000668,0.0094016212,-0.0032117339,9.8142997663,0.0040292662,-0.0846729855,9.7364006196,0.0294311619,0.0260442424,9.8305920166,0.1092305998,0.1559373809,-0.1138826696,0.0831790087,0.3007097942,0.0089319741,0.0489397747,0.0245629287,0.2995932974,-0.9696247329,1.5337700321,0.6763226357,-3.0671480963,-2.3828632305,1.4247691911,29.6261119597,40.3608290652,40.1691376883
4,1577399005.4400000572,-0.0289056054,0.0135476777,9.7532476239,-0.0294895570,-0.0343947507,9.7723136445,-0.0064818630,-0.0002919758,9.7803137817,0.0481954435,-0.0576856660,0.2065519007,-0.1914791945,-0.1570538777,-0.1131383384,-0.0426129597,0.0245629287,-0.1886879526,-0.1762954060,0.8180106838,0.6011756762,-3.0671480963,-2.3828632305,-0.9351473346,29.4583820050,40.1691376883,39.9774463114


In [39]:
data_esquerda.head()

,timestamp,acc_x_dashboard,acc_y_dashboard,acc_z_dashboard,acc_x_above_suspension,acc_y_above_suspension,acc_z_above_suspension,acc_x_below_suspension,acc_y_below_suspension,acc_z_below_suspension,gyro_x_dashboard,gyro_y_dashboard,gyro_z_dashboard,gyro_x_above_suspension,gyro_y_above_suspension,gyro_z_above_suspension,gyro_x_below_suspension,gyro_y_below_suspension,gyro_z_below_suspension,mag_x_dashboard,mag_y_dashboard,mag_z_dashboard,mag_x_above_suspension,mag_y_above_suspension,mag_z_above_suspension,temp_dashboard,temp_above_suspension,temp_below_suspension
0,1577399005.4000000954,-0.0252588275,0.0592564911,9.8268810040,0.0734611144,0.0542783035,9.8624757753,0.0255381510,0.0071256002,9.7666896342,0.0385284424,0.2510070801,-0.0434875488,-0.1386316811,0.0537779273,0.1455167445,-0.1100812640,-0.2390543620,-0.0352405366,0.9038242367,0.0940372321,-0.8131454772,0.7228538266,3.1487179487,-5.8353144130,31.0158744421,42.1100128793,36.7186929044
1,1577399005.4100000858,-0.0156820209,0.0568622894,9.8388520123,0.0183944762,0.0219565811,9.8481105653,0.0195526469,-0.0168164164,9.7810548442,-0.0530242920,-0.0846862793,0.0480651855,-0.1233728921,-0.0225160180,0.4659513148,0.1645769392,-0.0864664714,-0.2488635835,-0.3615296947,-0.8463350885,-0.6505163818,-4.6453308009,-3.9733821734,-5.6123087666,31.0637972864,42.0860514572,36.6707700602
2,1577399005.4200000763,-0.0420182391,0.0353144745,9.8101215924,0.1045857359,0.0794174209,9.8002265321,0.0411004618,0.0095198018,9.8049968608,0.0080108643,0.1289367676,0.1396179199,-0.2759607827,-0.0835511743,0.3743985804,0.1035417829,-0.2390543620,-0.1267932710,-1.6268836260,0.4701861603,0.1626290954,3.4069461403,-4.4481888482,-6.1326552748,30.9200287537,41.9902057687,36.8624614371
3,1577399005.4300000668,-0.0096965167,0.0568622894,9.7993476849,0.0543075011,0.0171681778,9.8086062379,0.0387062602,-0.0108309123,9.7798577433,0.0843048096,-0.0541687012,-0.0587463379,-0.0623377358,0.0690367164,0.3133634242,-0.0948224749,-0.0254313151,-0.0047229585,0.0000000000,-1.5986329450,-0.6505163818,2.3333092148,-3.9733821734,-5.8353144130,30.8721059095,42.1579357235,36.7666157486
4,1577399005.4400000572,-0.0444124408,0.0520738861,9.8508230206,0.0734611144,0.0267449844,9.7355830873,0.0051874369,0.0167024068,9.7523244242,-0.0225067139,0.2510070801,-0.0434875488,0.0902501548,-0.1445863305,0.4659513148,-0.0795636858,-0.2390543620,0.0257946196,-0.9038242367,-1.0344095526,1.7889200499,3.9437646030,3.6235246235,-6.9503426448,31.1117201306,42.1339743014,36.8145385929


In [40]:
data_gps.head()

,timestamp,latitude,longitude,elevation,accuracy,bearing,speed_meters_per_second,satellites,provider,hdop,vdop,pdop,geoidheight,ageofdgpsdata,dgpsid,activity,battery,annotation,distance_meters,elapsed_time_seconds
0,1577398854.0929999352,-27.7176688693,-51.0988452291,1010.7376196344,8.0000000000,196.8202800000,1.0591824000,0,gps,1.0000000000,1.7000000000,2.0000000000,3.6000000000,nan,nan,nan,85,nan,0.0000000000,0.0000000000
1,1577398855.0920000076,-27.7176899184,-51.0989381528,1001.9054212869,6.0000000000,244.7717900000,1.1281757000,11,gps,0.9000000000,1.5000000000,1.8000000000,3.6000000000,nan,nan,nan,85,nan,9.4560590017,0.9990000725
2,1577398856.0829999447,-27.7177004171,-51.0989531781,998.9896134512,6.0000000000,270.3939800000,0.2207288100,11,gps,0.9000000000,1.6000000000,1.8000000000,3.6000000000,nan,nan,nan,85,nan,1.8839061173,0.9909999371
3,1577398857.0829999447,-27.7177001244,-51.0989532787,994.7213603738,8.0000000000,nan,0.0000000000,14,gps,0.8000000000,1.3000000000,1.6000000000,3.6000000000,nan,nan,nan,85,nan,0.0339211000,1.0000000000
4,1577398858.0829999447,-27.7177001273,-51.0989534652,993.1997883760,8.0000000000,nan,0.0000000000,14,gps,0.8000000000,1.3000000000,1.5000000000,3.6000000000,nan,nan,nan,85,nan,0.0183958214,1.0000000000


<h4>Processamento</h4>

In [41]:
# Mapeia dados do GPS com base no timestamp da amostra MPU
def mergeMPUGPS(data_mpu, data_gps, side):
    
    columns = ['timestamp', 'latitude', 'longitude', 'speed_meters_per_second']
    result = []
    
    indexMPU = 0
    indexGPS = 0
    minusIndex = 2
    timestampMPU = 0
    timestampGPS = 0
    
    while indexMPU < len(data_mpu):
        
        timestampMPU = data_mpu.loc[indexMPU, 'timestamp']
        
        while timestampGPS <= timestampMPU and indexGPS < len(data_gps):
            timestampGPS = data_gps.loc[indexGPS, 'timestamp']
            indexGPS += 1
            
        if indexGPS == len(data_gps) and timestampMPU >= timestampGPS:
            minusIndex = 1
            
        series = data_gps.loc[indexGPS - minusIndex, columns] 
        result.append(series)
        indexMPU += 1
        
        load_bar.update(1)
        
    data = pd.DataFrame(result)
    data = data.reset_index(drop=True)
    data = data.rename(columns={ 'timestamp': 'timestamp_gps'})
    data = data.rename(columns={ 'speed_meters_per_second': 'speed'})
    
    series = data['timestamp_gps']
    data_mpu.insert(len(data_mpu.columns), series.name, series, True)
    series = data['latitude']
    data_mpu.insert(len(data_mpu.columns), series.name, series, True)
    series = data['longitude']
    data_mpu.insert(len(data_mpu.columns), series.name, series, True)
    series = data['speed']
    data_mpu.insert(len(data_mpu.columns), series.name, series, True)
    
    file = saveFolder + "dataset_gps_mpu_" + translateSide(side) + ".csv"
    data_mpu.to_csv(file, index=False)
    print("Salvo em ", file)

In [42]:
load_bar = tqdm_notebook(total=len(data_direita))
mergeMPUGPS(data_direita.copy(), data_gps.copy(), "direita")

Salvo em  E:\Dados Processados\100 Hz\2019-12-26 19-23-31 - Palio Cenário 3\dataset_gps_mpu_right.csv


In [43]:
load_bar = tqdm_notebook(total=len(data_esquerda))
mergeMPUGPS(data_esquerda.copy(), data_gps.copy(), "esquerda")

Salvo em  E:\Dados Processados\100 Hz\2019-12-26 19-23-31 - Palio Cenário 3\dataset_gps_mpu_left.csv
